# Commercial Bank Customer Retention Prediction

## APSTA-GE.2401: Statistical Consulting

## Scripts

Created on: 12/08/2020

Modified on: 12/09/2020

## Data Processing

----

### Description

This script processes data from the proprocess step.

### Data

The data are preprocessed feature sets:

  - `X_train.csv`: contains all features in Q3 and Q4 of 2019 for training. Imported as `X`.
  - `y_train.csv`: contains the label variable for validation. Imported as `y`.
  - `X_test.csv`: contains all features in Q1 of 2020 for testing. Imported as `X_true`.
   
After importing the data, we confirmed that both train sets have the same number of records: **145296**. We also confirmed that the testing set has **76722** records.

### Procedures

We first inspected the feature set. 

1. There are 55 features in the feature set. 

2. We checked if there are any missing values in the set. We found multiple columns that contain missing values, ranging from 0.005% to 100%. For columns containing a large portion of missing values, we dropped the column to reduce computational burden. For columns containing a small portion of missing values, we applied a deep learning library, [Datawig](https://github.com/awslabs/datawig), which learns machine learning models using deep neural networks to impute missing values in the data.

    - After dropping columns containing large portion of missing values, we reduced number of features to 45.

3. We then performed dummy coding to 

In [135]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

print('SUCCESS! All modules are imported.')

SUCCESS! All modules are imported.


In [136]:
X = pd.read_csv('../data/preprocess/X_train.csv')
y = pd.read_csv('../data/preprocess/y_train.csv')
X_true = pd.read_csv('../data/preprocess/X_test.csv')

In [137]:
print('The proprocessed training set has {} rows and {} columns.'.format(X.shape[0], X.shape[1]))
print('The proprocessed validation set has {} rows and {} columns.'.format(y.shape[0], y.shape[1]))
print('The proprocessed testing set has {} rows and {} columns.'.format(X_true.shape[0], X_true.shape[1]))

The proprocessed training set has 145296 rows and 56 columns.
The proprocessed validation set has 145296 rows and 2 columns.
The proprocessed testing set has 76722 rows and 56 columns.


### Functions

In [138]:
def check_missing(dat):
    '''Print missing values in each column of the dat
    @Param df dat: input data frame
    '''
    missing_val = dat.isnull().sum()
    for index in missing_val.index:
        if missing_val[index] > 0:
            print('{} has {} missing values. ({:.4%})'.format(index, missing_val[index], missing_val[index]/len(X)))

In [139]:
def code_datatime(dat, col, fmt):
    '''Convert col in dat to float using day0 as the reference date
    @Param df dat: input data frame
    @Param str col: column name
    @Param str fmt: date time format
    '''
    year = col + '_y'
    month = col + '_m'
    day = col + '_d'
    
    dat[year] = pd.DatetimeIndex(dat[col]).year
    dat[month] = pd.DatetimeIndex(dat[col]).month
    dat[day] = pd.DatetimeIndex(dat[col]).day
    dat.drop(col, axis=1)

----

## y (Label for Validation)

We applied `LabelBinarizer` to make the label binary. Originally, the label column contains three values: 
- 1: indicating churn
- 0: indicating no preference
- -1: indicating not churn

In [117]:
y['label'].value_counts()

 1    92818
 0    30237
-1    22241
Name: label, dtype: int64

In [118]:
y = pd.read_csv('../data/preprocess/y_train.csv')

In [120]:
lb = LabelBinarizer()
lb.fit(y['label'])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [121]:
lb.classes_

array([-1,  0,  1])

In [122]:
y_label = lb.transform(y['label'])

## X (Feature for Training)

### Missing Values

We first processed missing values in the data. Multiple columns contain missing values. The percentage of missing values in each column ranges from 0.0048% to 100.00%. We removed columns containing large portion of missing values.

In [140]:
# Check missing values
check_missing(X)

B6 has 8878 missing values. (6.1103%)
E2 has 6370 missing values. (4.3842%)
E3 has 6370 missing values. (4.3842%)
E4 has 84483 missing values. (58.1454%)
E5 has 55129 missing values. (37.9425%)
E6 has 7538 missing values. (5.1880%)
E7 has 142402 missing values. (98.0082%)
E8 has 127381 missing values. (87.6700%)
E9 has 145227 missing values. (99.9525%)
E10 has 816 missing values. (0.5616%)
E11 has 145296 missing values. (100.0000%)
E12 has 121324 missing values. (83.5013%)
E13 has 127502 missing values. (87.7533%)
E14 has 90010 missing values. (61.9494%)
E16 has 68530 missing values. (47.1658%)
E18 has 62147 missing values. (42.7727%)
C1 has 7 missing values. (0.0048%)
C2 has 7 missing values. (0.0048%)
I1 has 64 missing values. (0.0440%)
I5 has 11604 missing values. (7.9865%)
I9 has 145296 missing values. (100.0000%)
I10 has 128487 missing values. (88.4312%)
I13 has 143108 missing values. (98.4941%)
I14 has 129650 missing values. (89.2316%)


In [141]:
# Drop columns with large portion of missing values
col_to_drop = ['E7', 'E8', 'E9', 'E11', 'E12', 'E13', 'I9', 'I10', 'I13', 'I14']
X = X.drop(col_to_drop, axis=1)

In [142]:
print('After dropping columns containing large portion of missing values, now the set has {} columns.'.format(X.shape[1]))

After dropping columns containing large portion of missing values, now the set has 46 columns.


In [143]:
check_missing(X)

B6 has 8878 missing values. (6.1103%)
E2 has 6370 missing values. (4.3842%)
E3 has 6370 missing values. (4.3842%)
E4 has 84483 missing values. (58.1454%)
E5 has 55129 missing values. (37.9425%)
E6 has 7538 missing values. (5.1880%)
E10 has 816 missing values. (0.5616%)
E14 has 90010 missing values. (61.9494%)
E16 has 68530 missing values. (47.1658%)
E18 has 62147 missing values. (42.7727%)
C1 has 7 missing values. (0.0048%)
C2 has 7 missing values. (0.0048%)
I1 has 64 missing values. (0.0440%)
I5 has 11604 missing values. (7.9865%)


In [144]:
# B6: Latest transfer time
fmt = '%Y-%m-%d %H:%M:%S'
X['B6'] = pd.to_datetime(X['B6'], format=fmt, errors='ignore')

In [145]:
X['B6'] = X['B6'].replace(np.NaN, X['B6'].min())

In [146]:
# E category
fmt = '%Y-%m-%d'
col_names = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E10', 'E14', 'E16', 'E18']
for col_name in col_names:
    X[col_name] = pd.to_datetime(X[col_name], format=fmt, errors='ignore')

In [147]:
# E2: Online banking opening date
X['E2'] = X['E2'].replace(np.NaN, X['E2'].max())

In [148]:
# E3: Mobile banking opening date
X['E3'] = X['E3'].replace(np.NaN, X['E3'].max())

In [149]:
# E4: First online banking login date
X['E4'] = X['E4'].replace(np.NaN, X['E4'].max())

In [150]:
# E5: First mobile banking login date
X['E5'] = X['E5'].replace(np.NaN, X['E5'].max())

In [151]:
# E6: First demand deposit date
X['E6'] = X['E6'].replace(np.NaN, X['E6'].mean())

In [152]:
# E10: First cash transaction date
X['E10'] = X['E10'].replace(np.NaN, X['E10'].mean())

In [153]:
# E14: First transfer via mobile banking date
X['E14'] = X['E14'].replace(np.NaN, X['E14'].mean())

In [154]:
# E16: Maximum amount transferred out of another bank date
X['E16'] = X['E16'].replace(np.NaN, X['E16'].mean())

In [155]:
# E18: Maximum transfer amount from other bank date
X['E18'] = X['E18'].replace(np.NaN, X['E18'].mean())

In [156]:
# C1
X['C1'] = X['C1'].fillna(0)
# C2
X['C2'] = X['C2'].fillna(0)
# I1
X['I1'] = X['I1'].replace(np.NaN, '女性')
# I5
X['I5'] = X['I5'].replace(np.NaN, '未知')

In [157]:
check_missing(X)

### Drop Meaningless Columns

Based on the codebook, after mining into the data, we determined that the following columns contain meaningless information and, therefore, we dropped these columns:

- `I8`: constellation. We don't believe constellation can alter customer behavior.
- `I12`: field description. Contain only 1 different values.
- `I15`: QR code recipient.

In [158]:
col_to_drop = ['I8', 'I12', 'I15']
X = X.drop(col_to_drop, axis=1)

In [159]:
print('After dropping columns containing large portion of missing values, now the set has {} columns.'.format(X.shape[1]))

After dropping columns containing large portion of missing values, now the set has 43 columns.


In [160]:
X.columns

Index(['cust_no', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'B1', 'B2',
       'B3', 'B4', 'B5', 'B6', 'B7', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E10',
       'E14', 'E15', 'E16', 'E17', 'E18', 'C1', 'C2', 'I1', 'I2', 'I3', 'I4',
       'I5', 'I6', 'I7', 'I11', 'I16', 'I17', 'I18', 'I19', 'I20'],
      dtype='object')

### Dummy Coding

We dummy coded categorical columns.


#### Date Time Columns

To dummy code columns containing date and time, we first converted all string-like inputs as date time format. Then, we created dummy columns for `year`, `month` and `day` of each datetime column. Finally, we dropped the original column.

In [161]:
# B6: Latest transfer time
fmt = '%Y-%m-%d %H:%M:%S'
code_datatime(X, 'B6', fmt)

In [162]:
# E category
fmt = '%Y-%m-%d'
col_names = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E10', 'E14', 'E16', 'E18']
for col_name in col_names:
    code_datatime(X, col_name, fmt)

In [163]:
X.shape

(145296, 76)

In [164]:
X.columns

Index(['cust_no', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'B1', 'B2',
       'B3', 'B4', 'B5', 'B6', 'B7', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E10',
       'E14', 'E15', 'E16', 'E17', 'E18', 'C1', 'C2', 'I1', 'I2', 'I3', 'I4',
       'I5', 'I6', 'I7', 'I11', 'I16', 'I17', 'I18', 'I19', 'I20', 'B6_y',
       'B6_m', 'B6_d', 'E1_y', 'E1_m', 'E1_d', 'E2_y', 'E2_m', 'E2_d', 'E3_y',
       'E3_m', 'E3_d', 'E4_y', 'E4_m', 'E4_d', 'E5_y', 'E5_m', 'E5_d', 'E6_y',
       'E6_m', 'E6_d', 'E10_y', 'E10_m', 'E10_d', 'E14_y', 'E14_m', 'E14_d',
       'E16_y', 'E16_m', 'E16_d', 'E18_y', 'E18_m', 'E18_d'],
      dtype='object')

#### Categorical Columns

To prevent algorithms from interpreting hierarchy in dummy coded categorical columns, we applied `One-hot encoding` to categorical columns.

In [ ]:
# I5: Occupation
X['I5'] = pd.get_dummies(X['I5'])

In [172]:
enc = OneHotEncoder(handle_unknown='ignore')

In [173]:
cat_col_names = ['I1', 'I3', 'I4', 'I6', 'I16', 'I17', 'I18', 'I19', 'I20']
enc_df = pd.DataFrame(enc.fit_transform(X[cat_col_names]).toarray())

In [174]:
X_encoded = X.join(enc_df)

In [175]:
X_encoded = X_encoded.drop(cat_col_names, axis=1)

In [176]:
print('After dummy coding, now the set has {} columns.'.format(X_encoded.shape[1]))

After dummy coding, now the set has 87 columns.


In [177]:
X_encoded.shape

(145296, 87)

----

## X_true (Features for Testing)

### Missing Values

Similar to the `X`, we first processed missing values in the data. Multiple columns contain missing values. The percentage of missing values in each column ranges from 0.0048% to 100.00%. We removed columns containing large portion of missing values.

### Drop Meaningless Columns

Based on the codebook, after mining into the data, we determined that the following columns contain meaningless information and, therefore, we dropped these columns:

- `I8`: constellation. We don't believe constellation can alter customer behavior.
- `I12`: field description. Contain only 1 different values.
- `I15`: QR code recipient.

In [178]:
# Check missing values
check_missing(X_true)

B6 has 13429 missing values. (9.2425%)
E2 has 2889 missing values. (1.9884%)
E3 has 2889 missing values. (1.9884%)
E4 has 44687 missing values. (30.7558%)
E5 has 28121 missing values. (19.3543%)
E6 has 1799 missing values. (1.2382%)
E7 has 74289 missing values. (51.1294%)
E8 has 67269 missing values. (46.2979%)
E9 has 76619 missing values. (52.7330%)
E10 has 589 missing values. (0.4054%)
E11 has 76722 missing values. (52.8039%)
E12 has 62490 missing values. (43.0088%)
E13 has 67701 missing values. (46.5952%)
E14 has 45441 missing values. (31.2748%)
E16 has 34876 missing values. (24.0034%)
E18 has 32254 missing values. (22.1988%)
C1 has 3 missing values. (0.0021%)
C2 has 3 missing values. (0.0021%)
I1 has 32 missing values. (0.0220%)
I5 has 3795 missing values. (2.6119%)
I9 has 76722 missing values. (52.8039%)
I10 has 67797 missing values. (46.6613%)
I13 has 74722 missing values. (51.4274%)
I14 has 68320 missing values. (47.0213%)


In [179]:
# Drop columns with large portion of missing values
col_to_drop = ['E7', 'E8', 'E9', 'E11', 'E12', 'E13', 'I9', 'I10', 'I13', 'I14',
              'I8', 'I12', 'I15']
X_true = X_true.drop(col_to_drop, axis=1)

In [180]:
print('After dropping columns containing large portion of missing values and meaningless columns, now the set has {} columns.'.format(X_true.shape[1]))

After dropping columns containing large portion of missing values and meaningless columns, now the set has 43 columns.


In [181]:
check_missing(X_true)

B6 has 13429 missing values. (9.2425%)
E2 has 2889 missing values. (1.9884%)
E3 has 2889 missing values. (1.9884%)
E4 has 44687 missing values. (30.7558%)
E5 has 28121 missing values. (19.3543%)
E6 has 1799 missing values. (1.2382%)
E10 has 589 missing values. (0.4054%)
E14 has 45441 missing values. (31.2748%)
E16 has 34876 missing values. (24.0034%)
E18 has 32254 missing values. (22.1988%)
C1 has 3 missing values. (0.0021%)
C2 has 3 missing values. (0.0021%)
I1 has 32 missing values. (0.0220%)
I5 has 3795 missing values. (2.6119%)


In [182]:
# B6: Latest transfer time
fmt = '%Y-%m-%d %H:%M:%S'
X_true['B6'] = pd.to_datetime(X_true['B6'], format=fmt, errors='ignore')

In [183]:
X_true['B6'] = X_true['B6'].replace(np.NaN, X_true['B6'].min())

In [184]:
# E category
fmt = '%Y-%m-%d'
col_names = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E10', 'E14', 'E16', 'E18']
for col_name in col_names:
    X_true[col_name] = pd.to_datetime(X_true[col_name], format=fmt, errors='ignore')

In [185]:
# E2: Online banking opening date
X_true['E2'] = X_true['E2'].replace(np.NaN, X_true['E2'].max())

In [186]:
# E3: Mobile banking opening date
X_true['E3'] = X_true['E3'].replace(np.NaN, X_true['E3'].max())

In [187]:
# E4: First online banking login date
X_true['E4'] = X_true['E4'].replace(np.NaN, X_true['E4'].max())

In [188]:
# E5: First mobile banking login date
X_true['E5'] = X_true['E5'].replace(np.NaN, X_true['E5'].max())

In [189]:
# E6: First demand deposit date
X_true['E6'] = X_true['E6'].replace(np.NaN, X_true['E6'].mean())

In [190]:
# E10: First cash transaction date
X_true['E10'] = X_true['E10'].replace(np.NaN, X_true['E10'].mean())

In [191]:
# E14: First transfer via mobile banking date
X_true['E14'] = X_true['E14'].replace(np.NaN, X_true['E14'].mean())

In [192]:
# E16: Maximum amount transferred out of another bank date
X_true['E16'] = X_true['E16'].replace(np.NaN, X_true['E16'].mean())

In [193]:
# E18: Maximum transfer amount from other bank date
X_true['E18'] = X_true['E18'].replace(np.NaN, X_true['E18'].mean())

In [194]:
# C1
X_true['C1'] = X_true['C1'].fillna(0)
# C2
X_true['C2'] = X_true['C2'].fillna(0)
# I1
X_true['I1'] = X_true['I1'].replace(np.NaN, '女性')
# I5
X_true['I5'] = X_true['I5'].replace(np.NaN, '未知')

In [198]:
check_missing(X_true)

### Dummy Coding

Before applying `Datawig`, we dummy coded categorical columns.

In [196]:
# B6: Latest transfer time
fmt = '%Y-%m-%d %H:%M:%S'
code_datatime(X_true, 'B6', fmt)

In [197]:
# E category
fmt = '%Y-%m-%d'
col_names = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E10', 'E14', 'E16', 'E18']
for col_name in col_names:
    code_datatime(X_true, col_name, fmt)

In [199]:
X_true['I5'] = pd.get_dummies(X_true['I5'])

In [200]:
enc_df = pd.DataFrame(enc.fit_transform(X_true[cat_col_names]).toarray())

In [201]:
X_true_encoded = X_true.join(enc_df)
X_true_encoded = X_true_encoded.drop(cat_col_names, axis=1)

In [202]:
print('After dummy coding, now the set has {} columns.'.format(X_true_encoded.shape[1]))

After dummy coding, now the set has 87 columns.


In [203]:
X_encoded.columns

Index(['cust_no',      'X1',      'X2',      'X3',      'X4',      'X5',
            'X6',      'X7',      'X8',      'B1',      'B2',      'B3',
            'B4',      'B5',      'B6',      'B7',      'E1',      'E2',
            'E3',      'E4',      'E5',      'E6',     'E10',     'E14',
           'E15',     'E16',     'E17',     'E18',      'C1',      'C2',
            'I2',      'I5',      'I7',     'I11',    'B6_y',    'B6_m',
          'B6_d',    'E1_y',    'E1_m',    'E1_d',    'E2_y',    'E2_m',
          'E2_d',    'E3_y',    'E3_m',    'E3_d',    'E4_y',    'E4_m',
          'E4_d',    'E5_y',    'E5_m',    'E5_d',    'E6_y',    'E6_m',
          'E6_d',   'E10_y',   'E10_m',   'E10_d',   'E14_y',   'E14_m',
         'E14_d',   'E16_y',   'E16_m',   'E16_d',   'E18_y',   'E18_m',
         'E18_d',         0,         1,         2,         3,         4,
               5,         6,         7,         8,         9,        10,
              11,        12,        13,        14, 

## Output

In [ ]:
X_encoded.to_csv('../data/X_train.csv', index=False)

In [ ]:
X_true_encoded.to_csv('../data/X_test.csv', index=False)

In [ ]:
y.to_csv('../data/y_train.csv', index=False)

----